In [5]:
from model import vae,decoder
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Input
from keras.models import Model
from keras.optimizers import adam
from keras.callbacks import ModelCheckpoint
import cv2
import numpy as np
import mdn
from sklearn.preprocessing import MinMaxScaler

# Set paths

In [6]:

#ENCODED_DATA_PATH = '/Users/brandonquach/Desktop/cs159/dancenet-master/lv.npy'

# Load encoded data

In [7]:
#data = np.load(ENCODED_DATA_PATH)
#print(data.shape)

# Normalize data

In [8]:
#data = np.array(data).reshape(-1,128)
#scaler = MinMaxScaler(feature_range=(0, 1))
#scaler = scaler.fit(data)
#data =  scaler.transform(data)

In [9]:
#numComponents = 24
#outputDim = 128

In [10]:
def data_generator(batch_size,limit):

	batch = []
	counter = 1
	while 1:
		for i in range(1,limit+1):
			if counter >= limit:
				counter = 1
			img = cv2.imread("imgs/{}.jpg".format(counter),cv2.IMREAD_GRAYSCALE)
			img = img.reshape(120,208,1)
			batch.append(img)
			if len(batch) == batch_size:
				batch_np = np.array(batch) / 255
				batch = []
				return batch_np
			counter += 1

In [11]:
import os
X = data_generator(batch_size = 15901, limit = len(os.listdir('imgs')))

In [12]:
data = X

# Gumbel Softmax VAE

In [13]:
import numpy as np
import matplotlib.pyplot as plt

from keras.layers import Input, Dense, Lambda
from keras.models import Model, Sequential
from keras import backend as K
from keras import objectives
from keras.datasets import mnist
from keras.activations import softmax
from keras.objectives import binary_crossentropy as bce

batch_size = 100
data_dim = 24960
M = 80
N = 240

nb_epoch = 5
epsilon_std = 0.01

anneal_rate = 0.0003
min_temperature = 0.5

tau = K.variable(5.0, name="temperature")
x = Input(shape = (data_dim,))
h = Dense(256, activation='relu')(Dense(512, activation='relu')(x))
logits_y = Dense(M*N)(h)

def sampling(logits_y):
    U = K.random_uniform(K.shape(logits_y), 0, 1)
    y = logits_y - K.log(-K.log(U + 1e-20) + 1e-20) # logits + gumbel noise
    y = softmax(K.reshape(y, (-1, N, M)) / tau)
    y = K.reshape(y, (-1, N*M))
    return y

z = Lambda(sampling, output_shape=(M*N,))(logits_y)
generator = Sequential()
generator.add(Dense(256, activation='relu', input_shape=(N*M, )))
generator.add(Dense(512, activation='relu'))
generator.add(Dense(data_dim, activation='sigmoid'))
x_hat = generator(z)

# x_hat = Dense(data_dim, activation='softmax')(Dense(512, activation='relu')(Dense(256, activation='relu')(z)))
def gumbel_loss(x, x_hat):
    q_y = K.reshape(logits_y, (-1, N, M))
    q_y = softmax(q_y)
    log_q_y = K.log(q_y + 1e-20)
    kl_tmp = q_y * (log_q_y - K.log(1.0/M))
    KL = K.sum(kl_tmp, axis=(1, 2))
    elbo = data_dim * bce(x, x_hat) - KL 
    return elbo

model = Model(x, x_hat)
model.compile(optimizer='adam', loss=gumbel_loss)

X = data[0:len(data)-1]
X = X.reshape((len(X), np.prod(X.shape[1:])))
y = data[1:len(data)]
y = y.reshape((len(y), np.prod(y.shape[1:])))

y = y[:10000]
y_train = y[:7500]
y_test = y[7500:]
X = X[:10000]
X_train = X[:7500]
X_test = X[7500:]

for e in range(nb_epoch):
    model.fit(X_train, y_train,
        shuffle=False,
        nb_epoch=1,
        batch_size=batch_size,
        validation_data = (X_test, y_test))
    K.set_value(tau, np.max([K.get_value(tau) * np.exp(- anneal_rate * e), min_temperature]))

/anaconda3/envs/cs155/lib/python3.6/site-packages/ipykernel_launcher.py:72: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 7500 samples, validate on 2500 samples
Epoch 1/1
7500/7500 [==============================] - 61s 8ms/step - loss: 5937.1014 - val_loss: 4197.4366
Train on 7500 samples, validate on 2500 samples
Epoch 1/1
7500/7500 [==============================] - 59s 8ms/step - loss: 4907.8913 - val_loss: 4182.8099
Train on 7500 samples, validate on 2500 samples
Epoch 1/1
7500/7500 [==============================] - 60s 8ms/step - loss: 4878.6928 - val_loss: 4159.9007
Train on 7500 samples, validate on 2500 samples
Epoch 1/1
7500/7500 [==============================] - 60s 8ms/step - loss: 4862.5289 - val_loss: 4144.6293
Train on 7500 samples, validate on 2500 samples
Epoch 1/1
7500/7500 [==============================] - 61s 8ms/step - loss: 4844.8563 - val_loss: 4145.8712


In [14]:
model.fit(X_train, y_train,
        shuffle=False,
        nb_epoch=5,
        batch_size=batch_size,
        validation_data = (X_test, y_test))


/anaconda3/envs/cs155/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """


Train on 7500 samples, validate on 2500 samples
Epoch 1/5
7500/7500 [==============================] - 58s 8ms/step - loss: 4837.7502 - val_loss: 4147.7492
Epoch 2/5
7500/7500 [==============================] - 60s 8ms/step - loss: 4845.2968 - val_loss: 4138.4457
Epoch 3/5
7500/7500 [==============================] - 60s 8ms/step - loss: 4855.6367 - val_loss: 4111.3856
Epoch 4/5
7500/7500 [==============================] - 60s 8ms/step - loss: 4831.7820 - val_loss: 4130.8536
Epoch 5/5
7500/7500 [==============================] - 60s 8ms/step - loss: 4870.9187 - val_loss: 4110.6536


# Generate Video

In [15]:
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video = cv2.VideoWriter("out.mp4", fourcc, 30.0, (208, 120))
lv_in = X[0]

for i in range(500):
    temp = model.predict(lv_in.reshape(1, 24960))
    print(temp)
    img = temp
    img = np.array(img).reshape(120,208,1)
    img = img * 255
    img = np.array(img).astype("uint8")
    img = cv2.cvtColor(img,cv2.COLOR_GRAY2RGB)
    lv_in = temp
    video.write(img)
video.release()


[[0.9996574  0.99960965 0.99956936 ... 0.9956768  0.997433   0.997727  ]]
[[0.9996574  0.99960965 0.99956936 ... 0.9956768  0.997433   0.997727  ]]
[[0.9996574  0.99960965 0.99956936 ... 0.9956768  0.997433   0.997727  ]]
[[0.9996574  0.99960965 0.99956936 ... 0.9956768  0.997433   0.997727  ]]
[[0.9996574  0.99960965 0.99956936 ... 0.9956768  0.997433   0.997727  ]]
[[0.9996574  0.99960965 0.99956936 ... 0.9956768  0.997433   0.997727  ]]
[[0.9996574  0.99960965 0.99956936 ... 0.9956768  0.997433   0.997727  ]]
[[0.9996574  0.99960965 0.99956936 ... 0.9956768  0.997433   0.997727  ]]
[[0.9996574  0.99960965 0.99956936 ... 0.9956768  0.997433   0.997727  ]]
[[0.9996574  0.99960965 0.99956936 ... 0.9956768  0.997433   0.997727  ]]
[[0.9996574  0.99960965 0.99956936 ... 0.9956768  0.997433   0.997727  ]]
[[0.9996574  0.99960965 0.99956936 ... 0.9956768  0.997433   0.997727  ]]
[[0.9996574  0.99960965 0.99956936 ... 0.9956768  0.997433   0.997727  ]]
[[0.9996574  0.99960965 0.99956936 ...

[[0.9996574  0.99960965 0.99956936 ... 0.9956768  0.997433   0.997727  ]]
[[0.9996574  0.99960965 0.99956936 ... 0.9956768  0.997433   0.997727  ]]
[[0.9996574  0.99960965 0.99956936 ... 0.9956768  0.997433   0.997727  ]]
[[0.9996574  0.99960965 0.99956936 ... 0.9956768  0.997433   0.997727  ]]
[[0.9996574  0.99960965 0.99956936 ... 0.9956768  0.997433   0.997727  ]]
[[0.9996574  0.99960965 0.99956936 ... 0.9956768  0.997433   0.997727  ]]
[[0.9996574  0.99960965 0.99956936 ... 0.9956768  0.997433   0.997727  ]]
[[0.9996574  0.99960965 0.99956936 ... 0.9956768  0.997433   0.997727  ]]
[[0.9996574  0.99960965 0.99956936 ... 0.9956768  0.997433   0.997727  ]]
[[0.9996574  0.99960965 0.99956936 ... 0.9956768  0.997433   0.997727  ]]
[[0.9996574  0.99960965 0.99956936 ... 0.9956768  0.997433   0.997727  ]]
[[0.9996574  0.99960965 0.99956936 ... 0.9956768  0.997433   0.997727  ]]
[[0.9996574  0.99960965 0.99956936 ... 0.9956768  0.997433   0.997727  ]]
[[0.9996574  0.99960965 0.99956936 ...

[[0.9996574  0.99960965 0.99956936 ... 0.9956768  0.997433   0.997727  ]]
[[0.9996574  0.99960965 0.99956936 ... 0.9956768  0.997433   0.997727  ]]
[[0.9996574  0.99960965 0.99956936 ... 0.9956768  0.997433   0.997727  ]]
[[0.9996574  0.99960965 0.99956936 ... 0.9956768  0.997433   0.997727  ]]
[[0.9996574  0.99960965 0.99956936 ... 0.9956768  0.997433   0.997727  ]]
[[0.9996574  0.99960965 0.99956936 ... 0.9956768  0.997433   0.997727  ]]
[[0.9996574  0.99960965 0.99956936 ... 0.9956768  0.997433   0.997727  ]]
[[0.9996574  0.99960965 0.99956936 ... 0.9956768  0.997433   0.997727  ]]
[[0.9996574  0.99960965 0.99956936 ... 0.9956768  0.997433   0.997727  ]]
[[0.9996574  0.99960965 0.99956936 ... 0.9956768  0.997433   0.997727  ]]
[[0.9996574  0.99960965 0.99956936 ... 0.9956768  0.997433   0.997727  ]]
[[0.9996574  0.99960965 0.99956936 ... 0.9956768  0.997433   0.997727  ]]
[[0.9996574  0.99960965 0.99956936 ... 0.9956768  0.997433   0.997727  ]]
[[0.9996574  0.99960965 0.99956936 ...

[[0.9996574  0.99960965 0.99956936 ... 0.9956768  0.997433   0.997727  ]]
[[0.9996574  0.99960965 0.99956936 ... 0.9956768  0.997433   0.997727  ]]
[[0.9996574  0.99960965 0.99956936 ... 0.9956768  0.997433   0.997727  ]]
[[0.9996574  0.99960965 0.99956936 ... 0.9956768  0.997433   0.997727  ]]
[[0.9996574  0.99960965 0.99956936 ... 0.9956768  0.997433   0.997727  ]]
[[0.9996574  0.99960965 0.99956936 ... 0.9956768  0.997433   0.997727  ]]
[[0.9996574  0.99960965 0.99956936 ... 0.9956768  0.997433   0.997727  ]]
[[0.9996574  0.99960965 0.99956936 ... 0.9956768  0.997433   0.997727  ]]
[[0.9996574  0.99960965 0.99956936 ... 0.9956768  0.997433   0.997727  ]]
[[0.9996574  0.99960965 0.99956936 ... 0.9956768  0.997433   0.997727  ]]
[[0.9996574  0.99960965 0.99956936 ... 0.9956768  0.997433   0.997727  ]]
[[0.9996574  0.99960965 0.99956936 ... 0.9956768  0.997433   0.997727  ]]
[[0.9996574  0.99960965 0.99956936 ... 0.9956768  0.997433   0.997727  ]]
[[0.9996574  0.99960965 0.99956936 ...

[[0.9996574  0.99960965 0.99956936 ... 0.9956768  0.997433   0.997727  ]]
[[0.9996574  0.99960965 0.99956936 ... 0.9956768  0.997433   0.997727  ]]
[[0.9996574  0.99960965 0.99956936 ... 0.9956768  0.997433   0.997727  ]]
[[0.9996574  0.99960965 0.99956936 ... 0.9956768  0.997433   0.997727  ]]
[[0.9996574  0.99960965 0.99956936 ... 0.9956768  0.997433   0.997727  ]]
[[0.9996574  0.99960965 0.99956936 ... 0.9956768  0.997433   0.997727  ]]
[[0.9996574  0.99960965 0.99956936 ... 0.9956768  0.997433   0.997727  ]]
[[0.9996574  0.99960965 0.99956936 ... 0.9956768  0.997433   0.997727  ]]
[[0.9996574  0.99960965 0.99956936 ... 0.9956768  0.997433   0.997727  ]]
[[0.9996574  0.99960965 0.99956936 ... 0.9956768  0.997433   0.997727  ]]
[[0.9996574  0.99960965 0.99956936 ... 0.9956768  0.997433   0.997727  ]]
[[0.9996574  0.99960965 0.99956936 ... 0.9956768  0.997433   0.997727  ]]
[[0.9996574  0.99960965 0.99956936 ... 0.9956768  0.997433   0.997727  ]]
[[0.9996574  0.99960965 0.99956936 ...

In [ ]:
temp = np.random.random((1, 24960))
print(temp)
for layer in model.layers:
    weights = layer.get_weights()
    for weight in weights:
        if weight.shape == (512,):
            continue
        print(temp.shape, weight.shape)
        temp = np.dot(temp, weight)
        print(temp)
    print('done')

In [ ]:
temp = model.predict(np.random.random((1, 24960)))
print(temp)

In [ ]:
model.predict(temp)

In [ ]:
model.predict(temp)

In [ ]:
from keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [ ]:
x_train.shape

In [ ]:
28*28

In [ ]:
import numpy as np
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))


In [ ]:
x_train.shape

In [ ]:
argmax_y = K.max(K.reshape(logits_y, (-1, N, M)), axis=-1, keepdims=True)
argmax_y = K.equal(K.reshape(logits_y, (-1, N, M)), argmax_y)
encoder = K.function([x], [argmax_y, x_hat])

In [ ]:
code, x_hat_test = encoder([X_test[:100]])
plt.imshow(X_test[1].reshape(120, 208), cmap='gray')
plt.show()
plt.imshow(code[1].reshape(N, M), cmap='gray')
plt.show()
plt.imshow(x_hat_test[1].reshape(120, 208), cmap='gray')
plt.show()

In [ ]:
x_hat_test

In [ ]:
data.shape